# Basics

This notebook provides a quick introduction on how to get started with loading a simulation output and creating some basic figures.

We first begin by downloading and unpacking the sample data

In [ ]:
!wget -O osyris_sample_data.tar.gz https://project.esss.dk/owncloud/index.php/s/LLwJuO5RcAqfTCW/download
!tar -xzf osyris_sample_data.tar.gz

## Reading a RAMSES output

Import the `osyris` module and load the output of your choice (this will be output number `71` in this example).

<div class="alert alert-warning">

**IMPORTANT**

The data loader searches for descriptor files (`hydro_file_descriptor.txt`, `rt_file_descriptor` and the like)
inside the output directory to get the variable names.

If your version of RAMSES does not support this (the file descriptor did not exist in old versions of RAMSES),
you will need to generate one and copy it into each of your output directories.

</div>

In [ ]:
import osyris
import numpy as np

In [ ]:
data = osyris.load(71, scale="au")

In the call to `load`, the first argument is the output number. 
**Note** that you can use `-1` to select the last output in the directory. 

The second argument is the spatial scale you want to convert distances to.
Any length scale is allowed, e.g. `"cm"`, `"au"`, or `"pc"`, etc...

The `data` object has a `__str__` representation, which will return a list of all the variables contained in the loaded file, along with their minimum and maximum values:

In [ ]:
data

The `data` object behaves very much like a Python dictionary, and accessing a single variable is achieved by simply doing

In [ ]:
data["density"]

The information printed is the name of the variable, its minimum and maximum value, its physical unit, and the number of elements (or cells) in the variable.

## Creating a 2D histogram

We now wish to plot a 2d histogram of the logarithm of density versus logarithm of gas temperature,
for all the cells inside the computational domain.
We also use a logarithmic colormap which represents the cell counts in the $(\rho,T)$ plane

In [ ]:
osyris.histogram(data["density"], data["temperature"],
                      norm="log", loglog=True)

## Simple cut plane

The goal here is to create a 2D gas density slice, 100 au wide, through the plane normal to `z`,
that passes through the center of the young star forming system.

We first need to define where the center of the system is.
A simple definition is to use the coordinate of the cell with the highest density in the simulation.

In [ ]:
ind = np.argmax(data["density"])
center = data["xyz"][ind.values]
center

Once we have this center coordinate, we can use it as the origin for our cut plane:

In [ ]:
osyris.plane({"data": data["density"], "norm": "log"},
             dx=100,
             origin=center,
             direction="z")

where the first argument is the variable to display, `dx` is the extent of the vieweing region,
and `direction` is the normal to the slice plane.

<div class="alert alert-info">

**Note**

The units of `dx` are consistent with the `scale` specified when reading in the snapshot at the start of the notebook (`au` in this case).

</div>